In [ ]:
# Standard Libraries
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Deep Learning and PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
# Image Processing
from PIL import Image
from torchvision import transforms, models
import cv2

# File Handling
import h5py

# Metrics and Evaluation
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc

# Progress Visualization
from tqdm import tqdm

#sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

#Visualization
import plotly.express as px
import plotly.graph_objects as go


# 1) Problem Statement 

#### Skin cancer is the most common form of cancer in the United States and ranks 17th globally (WCRF).There are three major types of skin cancer—Basal Cell Carcinoma (BCC), Squamous Cell Carcinoma (SCC), and Melanoma. While BCC and SCC are considered less lethal, melanoma is the deadliest form ofskin cancer. It is expected to be diagnosed over 200,000 times in the US in 2024, with nearly 9,000 deathsprojected. Automated image analysis tools play a significant role in expediting clinical presentation anddiagnosis, positively impacting hundreds of thousands of people each year.For a telehealth app company, addressing the challenge of skin cancer detection in underserved populations or non-clinical settings is particularly significant. Current diagnostic methods rely on high-quality dermatoscope images, which are typically captured in dermatology clinics. These images reveal morphological features not visible to the naked eye. To provide this early detection service on our platform, we need to develop an algorithm capable of accurately classifying lower-quality malignant skin lesions from benign ones. Additionally, this algorithm should assist in diagnosing users based on their type of lesions and personal information.


# 2) Data Ingestion
#### From the Data Ingestion to Data Preprocessing stage, I utilized the original dataset prior to resampling, which is too large to upload to GitHub. As a result, you may encounter an error stating "No such file exists." To address this limitation, I discussed the issue of data size constraints with the professor. Subsequently, after resampling the dataset, I proceeded with data preprocessing and used the resampled data for the subsequent stages, including Feature Engineering and Model Development. This approach allowed me to handle the imbalanced dataset effectively while aligning with the constraints of data storage and accessibility.

## Load Data


In [ ]:
data = pd.read_csv('../data/raw/train-metadata.csv')

# 3) Explolatory Data Analysis

## Missing Value Analysis

In [ ]:
def df_stats(df: pd.DataFrame, include_all: bool = False):
    """
    Print statistics and null value counts for a pandas DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.
        include_all (bool): If True, include all columns in the descriptive statistics; otherwise, include only numeric columns.

    Returns:
        None
    """
    if df.empty:
        print("The DataFrame is empty.")
        return

    # Print descriptive statistics
    print("Descriptive Statistics:")
    if include_all:
        print(df.describe(include='all'))
    else:
        print(df.describe(include=[np.number]))
    print("\n" + "-"*50 + "\n")  # Separator for clarity

    # Print the number of null values per column
    print("Null Value Counts:")
    print(df.isnull().sum())
    print("\n" + "-"*50 + "\n")  # Separator for clarity

    # Additional information: Percentage of null values per column
    print("Percentage of Null Values:")
    print(df.isnull().mean() * 100)
    print("\n" + "-"*50 + "\n")  # Separator for clarity

    # Number of rows and columns
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")
    print("\n" + "-"*50 + "\n")  # Separator for clarity

In [ ]:
df_stats(data)

#### In the application I am developing, users will input an image and provide personal information. To ensure transparency and a user-centric design, only metadata accessible to users will be used as predictors in the model. Consequently, I have selected the following metadata fields for inclusion: "age_approx", "sex", "anatom_site_general", and "clin_size_long_diam_mm". These fields are both relevant to the prediction task and available to users.

#### From the data analysis, "age_approx", "sex", and "anatom_site_general" were identified as having missing values. However, the percentage of missing data for these fields is manageable, allowing for imputation strategies such as using the median for numerical fields like "age_approx" and the mode for categorical fields like "sex" and "anatom_site_general." This ensures the completeness and reliability of the metadata while maintaining the model's predictive performance.

## Visualize Target Variable

In [ ]:
# Target Distribution

# Count the occurrences of each target value and sort by index
target_counts = data['target'].value_counts().sort_index()

# Calculate the total number of samples in the training DataFrame
total = len(data)

# Create a list of percentages for each target class, formatted as a string
percentage = [f'{count/total:0.3%}' for count in target_counts]

# Create a bar plot to visualize the distribution of the target variable
fig = go.Figure(data=[
    go.Bar(
        x=target_counts.index,  # X-axis represents the unique target classes
        y=target_counts.values,  # Y-axis represents the counts of each class
        text=percentage,  # Display percentages on top of the bars
        textposition='auto'  # Automatically position text on bars
    )
])

# Update layout of the plot with titles and formatting
fig.update_layout(
    title='Distribution of Target Variable',  # Main title of the plot
    xaxis_title='Lesion Class',  # Title for the X-axis
    yaxis_title='Count',  # Title for the Y-axis
    template='plotly_white',  # Use a white background for the plot
    height=600, width=1200  # Set the dimensions of the plot
)

# Set the y-axis to a logarithmic scale to better visualize class distributions
fig.update_layout(yaxis=dict(type='log'))

# Add an annotation to show the total number of samples in the dataset
fig.add_annotation(
    text=f"<b>TOTAL SAMPLES:  {total:,}</b>",  # Format total count with commas
    xref="paper", yref="paper",  # Reference the entire paper for positioning
    x=0.98, y=1.05,  # Position the annotation near the top-right corner
    showarrow=False,  # Do not show an arrow pointing to the text
    font=dict(size=12)  # Set the font size for the annotation
)

# Display the plot
fig.show()

#### From the target distribution graph, it is evident that the dataset is highly imbalanced. Class 1, representing malignant cases, constitutes only 0.098% of the total data, while Class 0, representing benign cases, accounts for 99.902%. This extreme imbalance poses challenges for the model, as it may struggle to adequately learn patterns for the minority class, potentially leading to biased predictions heavily favoring the majority class. Addressing this imbalance is crucial to ensure the model's effectiveness and fairness, particularly for detecting malignant cases.

## Visualize categorical features

In [ ]:

def plot_categorical_feature_distribution(
    df: pd.DataFrame,
    feature_col: str,
    target_col: str = 'target',
    target_as_str: bool = True,
    log_y: bool = False,
    template_theme: str = "plotly_white",
    group_by_target: bool = True,
    stack_bar: bool = False
) -> None:
    """
    Plots the distribution of a categorical feature, optionally grouped by a target variable.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        feature_col (str): The name of the categorical feature column to plot.
        target_col (str, optional): The name of the target column. Defaults to 'target'.
        target_as_str (bool, optional): Whether to treat target variable as strings. Defaults to True.
        log_y (bool, optional): Whether to use a logarithmic scale for the Y-axis. Defaults to False.
        template_theme (str, optional): Plotly template theme to use. Defaults to 'plotly_white'.
        group_by_target (bool, optional): Whether to group bars by target variable. Defaults to True.
        stack_bar (bool, optional): Whether to stack bars instead of grouping. Defaults to False.

    Returns:
        None; displays the plot.
    """
    
    # Create a copy of the DataFrame and sort it based on feature and target columns
    _df = df.copy().sort_values(by=[feature_col, target_col]).reset_index(drop=True)

    # Check if we need to group the bars by the target variable
    if group_by_target:
        # Create a histogram plot grouped by the target variable
        fig = px.histogram(
            _df, x=feature_col, color=target_col,
            log_y=log_y, height=500, width=1200, template=template_theme,
            title=f'Distribution of {feature_col.upper()} By TARGET',
            barmode='group' if not stack_bar else 'stack'  # Choose between grouped or stacked bars
        )
    else:
        # Create a histogram plot without grouping by the target variable
        fig = px.histogram(
            _df, x=feature_col, color=feature_col, 
            log_y=log_y, height=500, width=1200, template=template_theme,
            title=f'<b>DISTRIBUTION OF {feature_col.replace("_", " ").upper()}',
        )  

    # Update the layout of the plot with titles and gaps
    fig.update_layout(
        bargap=0.1,  # Set the gap between bars
        xaxis_title=f"{feature_col.title()}",  # Format the X-axis title
        yaxis_title="Count",  # Title for the Y-axis
        showlegend=group_by_target  # Show legend only when grouped by target
    )

    # Apply log scale to the Y-axis if requested
    if log_y:
        fig.update_layout(yaxis_type="log")

    # Display the plot
    fig.show()


### Age_approx

In [ ]:
plot_categorical_feature_distribution(data, "age_approx", group_by_target=False)

In [ ]:
plot_categorical_feature_distribution(data, "age_approx", group_by_target=True, stack_bar=False, log_y=True)

### Anatom_Site_General

In [ ]:
plot_categorical_feature_distribution(data, "anatom_site_general", group_by_target=True, stack_bar=False, log_y = True)

### Sex

In [ ]:
plot_categorical_feature_distribution(data, "sex", group_by_target=True, stack_bar=False, log_y=True)

#### From these graphs, I found out that age groups under 40 and females, in particular, are underrepresented for malignant cases. This could lead to lower recall for these subgroups, as the model may not learn enough from the available data.

## Visualize continuous features

In [ ]:
def plot_continuous_feature_distribution(
    df: pd.DataFrame, 
    feature_col: str,
    plot_style: str = "histogram",
    feature_readable_name: str | None = None,
    target_col: str = "target",
    log_y: bool = False, 
    template_theme: str = "plotly_white",
    group_by_target: bool = True,
    n_bins: int = 50
) -> None:
    """
    Plots the distribution of a continuous feature in the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the feature and target columns.
        feature_col (str): The name of the feature column to plot.
        plot_style (str, optional): The style of the plot ('histogram' or 'box'). Defaults to 'histogram'.
        feature_readable_name (str | None, optional): A readable name for the feature to use in the title. Defaults to None.
        target_col (str, optional): The name of the target column. Defaults to 'target'.
        log_y (bool, optional): Whether to apply a logarithmic scale to the y-axis. Defaults to False.
        template_theme (str, optional): The Plotly template theme to use for the plot. Defaults to 'plotly_white'.
        group_by_target (bool, optional): Whether to group the plot by the target variable. Defaults to True.
        n_bins (int, optional): The number of bins to use for the histogram. Defaults to 50.

    Raises:
        TypeError: If df is not a pandas DataFrame.
        ValueError: If feature_col or target_col are not found in the DataFrame or if plot_style is invalid.

    Returns:
        None: Displays the plot.
    """
    
    # Input validation
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input 'df' must be a pandas DataFrame.")
    if feature_col not in df.columns:
        raise ValueError(f"Feature column '{feature_col}' not found in DataFrame.")
    if target_col not in df.columns:
        raise ValueError(f"Target column '{target_col}' not found in DataFrame.")
    if plot_style not in ["histogram", "box"]:
        raise ValueError("Invalid plot_style. Choose either 'histogram' or 'box'.")
    
    # Make a copy of the DataFrame to avoid modifying the original data
    _df = df.copy().sort_values(by=[feature_col, target_col]).reset_index(drop=True)

    # Plotting logic based on the chosen plot style
    if plot_style == "histogram":
        if group_by_target:
            # Create a histogram for each target value
            fig = go.Figure()
            for target_value in _df[target_col].unique():
                subset = _df[_df[target_col] == target_value]
                fig.add_trace(go.Histogram(
                    x=subset[feature_col],
                    name=str(target_value),
                    opacity=0.7,
                    nbinsx=n_bins
                ))

            # Update layout for overlay histogram
            fig.update_layout(
                barmode='overlay',
                title=f"Distribution of {feature_readable_name or feature_col.upper()} by Target",
                height=500, width=1200, template=template_theme,
                xaxis_title=feature_readable_name or feature_col,
                yaxis_title="Count",
                showlegend=True
            )
        else:
            # Create a single histogram without grouping
            fig = px.histogram(
                _df, x=feature_col, log_y=log_y, height=500, width=1200, template=template_theme,
                title=f"Distribution of {feature_readable_name or feature_col.upper()}",
                nbins=n_bins
            )

            # Update layout for single histogram
            fig.update_layout(
                xaxis_title=feature_readable_name or feature_col,
                yaxis_title="Count",
                showlegend=False
            )

    elif plot_style == "box":
        if group_by_target:
            # Create a box plot for each target value
            fig = go.Figure()
            for target_value in _df[target_col].unique():
                subset = _df[_df[target_col] == target_value]
                fig.add_trace(go.Box(
                    y=subset[feature_col],
                    name=str(target_value),
                    boxpoints='outliers',  # Show outliers
                    boxmean=True  # Show mean in the box plot
                ))

            # Update layout for box plot grouped by target
            fig.update_layout(
                title=f'Distribution of {feature_readable_name or feature_col.upper()} by Target (includes likely outliers)', 
                height=500, width=1200, template=template_theme,
                xaxis_title='Target',
                yaxis_title=f'{feature_readable_name or feature_col}',
                showlegend=True
            )
        else:
            # Create a single box plot without grouping
            fig = px.box(
                _df, y=feature_col,
                height=500,
                width=1200,
                template=template_theme,
                title=f"Distribution of {feature_readable_name or feature_col.upper()}",
                points="outliers",  # Show outliers
            )
                             
            # Update layout for single box plot
            fig.update_layout(
                yaxis_title=f'{feature_readable_name or feature_col}',
                showlegend=False
            )   

    # Apply log scale to y-axis if requested (only for histogram)
    if log_y and plot_style == "histogram":
        fig.update_layout(yaxis_type='log')
    
    # Display the plot
    fig.show()

### clin_size_long_diam_mm

In [ ]:
plot_continuous_feature_distribution(data, 'clin_size_long_diam_mm', plot_style="box", log_y=True, group_by_target=True)

#### The boxplot above shows significant outliers in the "clin_size_long_diam_mm" feature for both classes, especially Class 0. These outliers can negatively impact the training of a neural network by skewing the weight updates

In [ ]:
plot_continuous_feature_distribution(data, 'clin_size_long_diam_mm', plot_style="histogram", log_y=True, group_by_target=True, n_bins=100)

## Visualize Images

In [ ]:
#Load image from hdf5 file
def load_image_from_hdf5(isic_id: str,
                         file_path: str = "../data/raw/train-image.hdf5",
                         n_channels: int = 3):
    # Handle the case where the isic_id is passed incorrectly
    if not isic_id.lower().startswith("isic"):
        isic_id = f"ISIC_{int(str(isic_id).split('_', 1)[-1]):>07}"
        
    # Open the HDF5 file in read mode
    with h5py.File(file_path, 'r') as hf:
        
        # Retrieve the image data from the HDF5 dataset using the provided ISIC ID
        try:
            image_data = hf[isic_id][()]
        except KeyError:
            raise KeyError(f"ISIC ID {isic_id} not found in HDF5 file.")

        # Convert the binary data to a numpy array
        image_array = np.frombuffer(image_data, np.uint8)

        # Decode the image from the numpy array
        if n_channels == 3:
            # Load the image as a color image (BGR) and convert to RGB
            image = cv2.cvtColor(cv2.imdecode(image_array, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
        else:
            # Load the image as a grayscale image
            image = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)

        # If the image failed to load for some reason (problems decoding) ...
        if image is None:
            raise ValueError(f"Could not decode image for ISIC ID: {isic_id}")
        
        return image

In [ ]:
def plot_images_by_target(df: pd.DataFrame, target_value: int, max_images: int = 10) -> None:
    """Load and plot images based on the target value.

    Args:
        processed_df (pd.DataFrame): The DataFrame containing image metadata.
        target_value (int): The target value to filter images.
        max_images (int, optional): Maximum number of images to display. Defaults to 10.
    
    Returns:
        None; displays a plot of the images.
    """
    # Validate inputs
    if not isinstance(target_value, int):
        raise ValueError("target_value must be an integer.")
    if not isinstance(max_images, int) or max_images <= 0:
        raise ValueError("max_images must be a positive integer.")

    # Filter the DataFrame for the specified target value and limit the number of images
    filtered_df = df[df['target'] == target_value].head(max_images)

    images = []  # Initialize a list to hold the loaded images
    for isic_id in filtered_df['isic_id']:
        try:
            # Load the image using the provided ISIC ID from the HDF5 file
            image = load_image_from_hdf5(isic_id)
            images.append(image)  # Append the loaded image to the list
        except Exception as e:
            print(f"Error loading image for ISIC ID {isic_id}: {e}")

    # Create a DataFrame to store the loaded images along with their metadata
    image_df = pd.DataFrame({
        'isic_id': filtered_df['isic_id'],
        'target': filtered_df['target'],
        'image': images
    })

    n_images = len(image_df)  # Get the number of images to display
    fig, axes = plt.subplots(1, n_images, figsize=(15, 5))  # Create a subplot for each image
    fig.suptitle(f'Images of Lesions with Target Value {target_value}', fontsize=14)  # Main title

    # Iterate over the axes, ISIC IDs, and images to display each image
    for ax, isic_id, img in zip(axes, image_df['isic_id'], image_df['image']):
        ax.imshow(img)  # Display the image
        ax.set_title(f'ISIC ID: {isic_id}', fontsize=5)  # Set the title for each image
        ax.axis('off')  # Hide the axis

    plt.tight_layout()  # Adjust layout to make room for the main title
    plt.show()  # Display the plot



In [ ]:
plot_images_by_target(data, target_value=1, max_images=10)

In [ ]:
plot_images_by_target(data, target_value=0, max_images=10)

## Correlation Analysis
#### To perform correlation analysis, I must first split the dataset into training, validation, and test sets. The training data is then used for the correlation analysis due to computational constraints that prevent me from using the entire dataset. By focusing on the training data, I ensure that the subset adequately represents the overall dataset while remaining manageable for analysis. This allows for meaningful computation of correlation coefficients and effective visualization of feature relationships using a heatmap.

### Split Data inot Train, Validation and Test
#### I split the data into 70% train, 15% validation and 15% validation

In [ ]:
## Load data from the "train-metadata.csv file"  and split into train, val, test

try:
    data = pd.read_csv('../data/raw/train-metadata.csv')
except FileNotFoundError:
    print("Error: The specified CSV file was not found.")
    raise  # Re-raise the error after logging
except pd.errors.EmptyDataError:
    print("Error: The CSV file is empty.")
    raise
except pd.errors.ParserError:
    print("Error: The CSV file could not be parsed.")
    raise

# Select features (X) and the target variable (y)
try:
    X = data[['isic_id', 'age_approx', 'sex', 'anatom_site_general', 'clin_size_long_diam_mm']]
    y = data['target']
except KeyError as e:
    print(f"Error: Missing expected column in the dataset: {e}")
    raise

# Split the data into training and temporary sets (70% train, 30% temp)
try:
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, 
        test_size=0.3, 
        random_state=88, 
        stratify=y  # Ensures the target variable distribution is preserved
    )
except ValueError as e:
    print(f"Error during train-test split: {e}")
    raise

# Further split the temporary set into validation and test sets (15% val, 15% test)
try:
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, 
        test_size=0.5,  # This effectively splits the 30% temp into two equal parts
        random_state=88, 
        stratify=y_temp  # Again preserves the target variable distribution
    )
except ValueError as e:
    print(f"Error during validation-test split: {e}")
    raise

# Create DataFrames for the training, validation, and test sets
train_df = pd.concat([X_train, y_train], axis=1)
validation_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Save the processed DataFrames to CSV files
try:
    train_df.to_csv('../data/processed/train-metadata.csv', index=False)
    validation_df.to_csv('../data/processed/validation-metadata.csv', index=False)
    test_df.to_csv('../data/processed/test-metadata.csv', index=False)
except Exception as e:
    print(f"Error while saving CSV files: {e}")
    raise

### Preprocessing Pipeline

In [ ]:
# Custom transformer for handling missing values
class MissingValueHandler(BaseEstimator, TransformerMixin):
    # Fit method, not modifying any parameters, just returning self
    def fit(self, X, y=None):
        return self

    # Transform method to handle missing values
    def transform(self, X):
        # Ensure input is a pandas DataFrame
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")

        # Identify numerical columns
        num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        # Identify categorical columns
        cat_cols = X.select_dtypes(include=['object', 'category']).columns

        # Create imputer for numerical data using median
        num_imputer = SimpleImputer(strategy="median")
        # Apply imputer to numerical columns
        X[num_cols] = num_imputer.fit_transform(X[num_cols])

        # Create imputer for categorical data using the most frequent value
        cat_imputer = SimpleImputer(strategy="most_frequent")
        # Apply imputer to categorical columns
        X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

        return X  # Return the transformed DataFrame

# Custom transformer for one-hot encoding
class OneHotEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Initialize the OneHotEncoder with specified parameters
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

    # Fit method to learn the categories for encoding
    def fit(self, X, y=None):
        # Ensure input is a pandas DataFrame
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")
        # Fit the encoder to categorical columns
        self.encoder.fit(X.select_dtypes(include=['object', 'category']))
        return self

    # Transform method to apply one-hot encoding
    def transform(self, X):
        # Ensure input is a pandas DataFrame
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")

        # Transform categorical columns to one-hot encoding
        encoded_cols = self.encoder.transform(X.select_dtypes(include=['object', 'category']))
        # Get the new column names after encoding
        new_columns = self.encoder.get_feature_names_out(X.select_dtypes(include=['object', 'category']).columns)

        # Create a DataFrame for the encoded columns
        encode_df = pd.DataFrame(encoded_cols, columns=new_columns, index=X.index)
        # Concatenate the original DataFrame (excluding categorical columns) with the encoded DataFrame
        return pd.concat([X.select_dtypes(exclude=['object', 'category']), encode_df], axis=1)

# Custom transformer for scaling numerical features
class NumericalScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Initialize the StandardScaler for scaling numerical features
        self.scaler = StandardScaler()

    # Fit method to learn the scaling parameters
    def fit(self, X, y=None):
        # Ensure input is a pandas DataFrame
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")
        # Identify numerical columns
        num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        # Fit the scaler to the numerical columns
        self.scaler.fit(X[num_cols])
        return self

    # Transform method to apply scaling
    def transform(self, X):
        # Ensure input is a pandas DataFrame
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")

        # Identify numerical columns
        num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        # Apply scaling to the numerical columns
        X[num_cols] = self.scaler.transform(X[num_cols])
        return X  # Return the scaled DataFrame

# Custom transformer for handling age approximation
class AgeApproxTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting required for this transformer

    # Transform method to round age approximations
    def transform(self, X):
        # Ensure input is a pandas DataFrame
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")
        # Check if 'age_approx' is in the DataFrame
        if 'age_approx' in X.columns:
            # Round the age and convert to integer type
            X['age_approx'] = X['age_approx'].round().astype('Int64')
        return X  # Return the transformed DataFrame

# Create the complete pipeline for preprocessing
def create_pipeline() -> Pipeline:
    # Define a pipeline with the specified transformers
    pipeline = Pipeline(steps=[
        ('age_transformer', AgeApproxTransformer()),  # Age approximation
        ('missing_value_handler', MissingValueHandler()),  # Handling missing values
        ('cat_encoder', OneHotEncoderTransformer()),  # One-hot encoding categorical features
        ('num_scaler', NumericalScaler())  # Scaling all numerical features (including encoded features)
    ])
    return pipeline  # Return the constructed pipeline



#### After week 4, I realized that applying StandardScaler to my dataset may not be the optimal choice for a neural network model. Instead, using MinMaxScaler is more appropriate, as it scales the data to a range of 0 to 1, which aligns better with the activation functions commonly used in neural networks. This adjustment ensures that the input features are normalized in a way that enhances the model's learning efficiency and stability. Moving forward, this is one of the changes I will implement to improve the overall performance of my model.

In [ ]:
# Load the training metadata from a CSV file


# Drop the 'target' and 'isic_id' columns to create the feature set
X = train_df.drop(columns=['target', 'isic_id'])

# Keep the 'target' and 'isic_id' columns in a separate DataFrame for later use
temp = train_df[['target', 'isic_id']]

# Create the preprocessing pipeline using the previously defined function
pipeline = create_pipeline()

try:
    # Fit the pipeline to the feature set and transform the data
    processed_X = pipeline.fit_transform(X)
except Exception as e:
    # Log any errors that occur during fitting and transformation
    print(f"Error occurred during pipeline processing: {e}")

# Concatenate the processed features with the target and ISIC ID columns
processed_df = pd.concat([processed_X, temp], axis=1)

# Calculate the correlation matrix, excluding the 'isic_id' column
correlation_matrix = processed_df.drop(columns=['isic_id']).corr()

# Set the size of the plot
plt.figure(figsize=(10, 8))

# Create the heatmap using seaborn
sns.heatmap(
    correlation_matrix,            # The correlation matrix to visualize
    annot=True,                    # Annotate each cell with the numeric value
    fmt=".2f",                     # Format the annotation to two decimal places
    cmap='coolwarm',               # Color map for the heatmap
    square=True                    # Ensure each cell is square-shaped
)

# Set the title for the plot
plt.title('Correlation Matrix Heatmap (Including Target Variable)')

# Display the plot
plt.show()


#### We can see that sex_female and sex_male are highly correlated, but I do not think it will significantly affect the accuracy of the neural network model because the relationship between these two features is binary and mutually exclusive. In this case, one being 1 automatically implies the other is 0. Neural networks are capable of learning such simple relationships efficiently without causing confusion or overfitting.

#### In the future, as a best practice, one of these features could be dropped without any loss of information, as retaining only sex_female (or sex_male) is sufficient to convey the same information. However, for interpretability and domain alignment, keeping both features might be beneficial depending on how the model's results are presented or used. This decision could also depend on how stakeholders prefer to view or analyze the results of the predictions.

# 4) Preprocess & Feature Engineer data
#### For metadata preprocessing, I will utilize a custom pipeline that includes handling missing values, one-hot encoding categorical variables, and scaling numerical features. This ensures that all metadata inputs are properly formatted for input into the neural network.

#### For image feature engineering, I will apply transformations such as resizing, rotation, and random cropping to the images. These transformations help improve model generalization by introducing variability in the training data, thereby reducing the risk of overfitting.

#### By combining these preprocessing steps, I aim to ensure that both the metadata and image inputs are in optimal condition for the neural network, leading to better model performance and robustness.

## Handle data imbalance in training set

In [ ]:
# Assuming 'train' is your DataFrame with the target column 'target'
try:
    # Print class distribution before sampling
    print("Class Distribution Before Sampling (%):")
    display(train_df.target.value_counts(normalize=True) * 100)

    # Check if the 'target' column exists in the DataFrame
    if 'target' not in train_df.columns:
        raise KeyError("The 'target' column is not found in the DataFrame.")

    # Sampling process
    try:
        # Sample the majority class (0) with a fraction of 0.01
        majority_df = train_df.query("target == 0").sample(frac=0.01, random_state=42)  # Fixed random seed for reproducibility
        
        # Sample the minority class (1) with a factor of 5.0, allowing replacement
        minority_df = train_df.query("target == 1").sample(frac=5.0, replace=True, random_state=42)
        
        # Combine the sampled data into a new balanced DataFrame
        train_balanced = pd.concat([majority_df, minority_df], axis=0).sample(frac=1.0, random_state=42)  # Shuffle the combined DataFrame
    except ValueError as e:
        raise ValueError(f"Error during sampling: {e}")

    # Print class distribution after sampling
    print("\nClass Distribution After Sampling (%):")
    display(train_balanced.target.value_counts(normalize=True) * 100)

except Exception as e:
    print(f"An error occurred: {e}")

#### As you can see, I have downsized the majority class and upsized the minority class to address the class imbalance in the dataset. This resampling strategy aims to create a more balanced distribution of classes, which can help the model better identify and classify minority class instances.

#### One important consideration is that this approach may still affect the model's ability to generalize to new image data and metadata. By artificially altering the class distribution, there is a risk of overfitting to the resampled data, especially if the model becomes too focused on the minority class. To mitigate this, I will employ strategies such as cross-validation, early stopping, and careful selection of evaluation metrics (e.g., AUROC, precision-recall) to ensure the model remains robust on unseen data.

## Metadata Preprocessing Pipeline
#### The metadata prerpocessing pipeline includes:
* #### Impute Missing Value for both categorical and numerical data
* #### One-Hot Encode for categorical data
* #### Scale numerical variables with StandardScaler
* #### Convert age from float to interger 

In [ ]:
#seperate case id and target variable from dependable variables
pipeline = create_pipeline()
X_train = train_balanced.drop(columns=['isic_id','target'])
temp_train = train_balanced[['target','isic_id']]
train_processed_df = pd.concat([pipeline.fit_transform(X_train),temp_train],axis=1)

# Process validation data
X_validation = validation_df.drop(columns=['isic_id', 'target'])
temp_validation = validation_df[['target', 'isic_id']]
validation_processed_df = pd.concat([pipeline.transform(X_validation), temp_validation], axis=1)

# Process test data
X_test = test_df.drop(columns=['isic_id', 'target'])
temp_test = test_df[['target', 'isic_id']]
test_processed_df = pd.concat([pipeline.transform(X_test), temp_test], axis=1)

# Save the processed dataframes
train_processed_df.to_csv('../data/processed/processed-train-metadata.csv', index=False)
validation_processed_df.to_csv('../data/processed/processed-validation-metadata.csv', index=False)
test_processed_df.to_csv('../data/processed/processed-test-metadata.csv', index=False)

## Feature Engineer Image Data

#### I will create a custom dataset to store and preprocess the data, enabling efficient data loading and feature engineering for later use in the model. This approach ensures that the data is preprocessed consistently and allows for easy access during model training and evaluation.

### Create Custom Dataset

In [ ]:

class MultiInputDataset(Dataset):
    def __init__(self, hdf5_file, csv_file, transform=None):
        # Open the HDF5 file with error handling
        try:
            self.hdf5_file = h5py.File(hdf5_file, 'r')  # Read-only mode
        except Exception as e:
            raise IOError(f"Could not open HDF5 file: {hdf5_file}. Error: {e}")

        # Read the CSV file containing image labels and additional features
        try:
            self.labels_df = pd.read_csv(csv_file)
        except Exception as e:
            raise IOError(f"Could not read CSV file: {csv_file}. Error: {e}")

        # Ensure that all image IDs from the CSV are present in the HDF5 file
        self.image_ids = self.labels_df['isic_id'].values
        for image_id in self.image_ids:
            if str(image_id) not in self.hdf5_file.keys():
                raise ValueError(f"Image id {image_id} not found in HDF5 file.")

        # Store any transformations to be applied to the images
        self.transform = transform

    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.labels_df)

    def __getitem__(self, idx):
        # Get the image ID from the CSV file based on index
        image_id = str(self.labels_df.iloc[idx]['isic_id'])

        # Load the image data from the HDF5 file
        image_bytes = self.hdf5_file[image_id][()]

        # Convert the image bytes to a PIL Image
        image = Image.open(io.BytesIO(image_bytes))

        # Apply any specified transformations to the image
        if self.transform:
            image = self.transform(image)

        # Retrieve the label
        label = torch.tensor(self.labels_df.iloc[idx]['target'], dtype=torch.long)  # Adjust dtype if needed

        # Retrieve other features, excluding 'isic_id' and 'target'
        other_variables = self.labels_df.iloc[idx].drop(['isic_id', 'target']).values.astype(float)

        # Convert other variables (metadata) to a tensor
        metadata_tensor = torch.tensor(other_variables, dtype=torch.float32)

        # Return the image, metadata, and label
        return image, metadata_tensor, label


In [ ]:
# Feature Engineer for train,validation and test image data

def get_train_transform(resize_size=(224, 224), crop_size=128, rotation_degree=10, normalize_means=(0.5, 0.5, 0.5), normalize_stds=(0.5, 0.5, 0.5)):
    """
    Returns the transformations for the training dataset, including data augmentation.

    Args:
        resize_size (tuple): The size to resize the image before cropping.
        crop_size (int): The size of the random crop.
        rotation_degree (int): Maximum degree for random rotation.
        normalize_means (tuple): Means for normalization.
        normalize_stds (tuple): Standard deviations for normalization.

    Returns:
        transforms.Compose: The composed transformations for the training set.
    """
    return transforms.Compose([
        transforms.Resize(resize_size),  # Resize to specified size
        transforms.RandomResizedCrop(crop_size, scale=(0.8, 1.0)),  # Random crop with scale
        transforms.RandomRotation(rotation_degree),  # Randomly rotate images
        transforms.ToTensor(),  # Convert image to PyTorch tensor
        transforms.Normalize(normalize_means, normalize_stds)  # Normalize with specified means and stds
    ])

def get_normal_transform(resize_size=(224, 224), normalize_means=(0.5, 0.5, 0.5), normalize_stds=(0.5, 0.5, 0.5)):
    """
    Returns the transformations for the validation/test dataset (without data augmentation).

    Args:
        resize_size (tuple): The size to resize the image.
        normalize_means (tuple): Means for normalization.
        normalize_stds (tuple): Standard deviations for normalization.

    Returns:
        transforms.Compose: The composed transformations for the validation/test set.
    """
    return transforms.Compose([
        transforms.Resize(resize_size),  # Resize to specified size
        transforms.ToTensor(),  # Convert image to PyTorch tensor
        transforms.Normalize(normalize_means, normalize_stds)  # Normalize with specified means and stds
    ])





# Model Development 
#### In this stage, I will use the resampled dataset to address size constraints and ensure efficient model development. The resampled dataset allows for faster computations while preserving the data's core characteristics, which is critical for iterative model development and evaluation.

#### I will develop three multi-input neural network models with slight variations in the image processing component. Each of these models will accept two inputs — image data and metadata — which will be processed independently before being combined for final prediction.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # this will deetct 

## Model Building

### CNN

In [ ]:
class CustomImageFeatureCNN2(nn.Module):
    def __init__(self, feature_input_size, input_image_size=(128, 128)):
        super(CustomImageFeatureCNN2, self).__init__()

        # Image CNN with Batch Normalization
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # Batch normalization after conv1

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # Batch normalization after conv2

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # Batch normalization after conv3

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 Max pooling

        # Dynamically calculate the flattened size of the feature map
        self.flattened_size = self._get_flattened_size(input_image_size)

        # Fully connected layer after the CNN layers
        self.fc_image = nn.Linear(self.flattened_size, 512)

        # Fully connected layer for metadata (feature data)
        self.fc_metadata = nn.Linear(feature_input_size, 128)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.5)  # 50% dropout

        # Final fully connected layer for binary classification (combined image + feature input)
        self.fc_combined = nn.Linear(512 + 128, 1)  # Change 2 to 1 for binary classification

    def _get_flattened_size(self, input_image_size):
        # Forward pass a dummy image to get the size of the flattened features
        dummy_image = torch.zeros(1, 3, *input_image_size)  # Batch size of 1, 3 channels (RGB), and input size
        x = self.pool(F.relu(self.bn1(self.conv1(dummy_image))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        return x.view(-1).shape[0]  # Flatten and return the size

    def forward(self, image, metadata):
        # Forward pass for the image through the CNN
        x = self.pool(F.relu(self.bn1(self.conv1(image))))  # Conv layer 1 with ReLU, BatchNorm, MaxPool
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Conv layer 2 with ReLU, BatchNorm, MaxPool
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Conv layer 3 with ReLU, BatchNorm, MaxPool

        # Flatten the feature map to feed into fully connected layer
        x = x.view(x.size(0), -1)  # Flatten feature maps into a 1D vector
        image_features = F.relu(self.fc_image(x))

        # Process metadata (feature data)
        metadata_features = F.relu(self.fc_metadata(metadata))

        # Ensure the batch sizes are consistent
        assert image_features.shape[0] == metadata_features.shape[0], \
            f"Batch sizes do not match! Image batch size: {image_features.shape[0]}, Metadata batch size: {metadata_features.shape[0]}"

        # Concatenate image features and metadata features
        combined_features = torch.cat((image_features, metadata_features), dim=1)

        # Dropout and final classification layer
        combined_features = self.dropout(combined_features)
        output = self.fc_combined(combined_features)

        # If you're using BCELoss, uncomment the next line to apply sigmoid
        output = torch.sigmoid(output)

        return output

### Resnet

In [ ]:
class CustomImageFeatureResNet(nn.Module):
    def __init__(self, feature_input_size, pretrained=True):
        super(CustomImageFeatureResNet, self).__init__()

        # Load a pretrained ResNet model for image feature extraction (ResNet18 in this case)
        resnet = models.resnet18(pretrained=pretrained)  # Change to resnet50, resnet101 as needed
        self.resnet = nn.Sequential(*list(resnet.children())[:-1])  # Remove the final classification layer

        # The output of ResNet18's last conv layer is 512-dimensional (for ResNet50, it would be 2048)
        self.fc_image = nn.Linear(resnet.fc.in_features, 512)  # Adjust if using ResNet50

        # Fully connected layer for metadata (feature data)
        self.fc_metadata = nn.Linear(feature_input_size, 128)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.5)  # 50% dropout

        # Final fully connected layer for binary classification (combined image + feature input)
        self.fc_combined = nn.Linear(512 + 128, 1)  # For binary classification

    def forward(self, image, metadata):
        # Forward pass for the image through the ResNet (without the final classification layer)
        x = self.resnet(image)  # ResNet feature extraction
        x = x.view(x.size(0), -1)  # Flatten the ResNet output
        image_features = F.relu(self.fc_image(x))

        # Process metadata (feature data)
        metadata_features = F.relu(self.fc_metadata(metadata))

        # Ensure the batch sizes are consistent
        assert image_features.shape[0] == metadata_features.shape[0], \
            f"Batch sizes do not match! Image batch size: {image_features.shape[0]}, Metadata batch size: {metadata_features.shape[0]}"

        # Concatenate image features and metadata features
        combined_features = torch.cat((image_features, metadata_features), dim=1)

        # Dropout and final classification layer
        combined_features = self.dropout(combined_features)
        output = self.fc_combined(combined_features)

        # If you're using BCELoss, uncomment the next line to apply sigmoid
        output = torch.sigmoid(output)

        return output

### EfficientNet

In [ ]:
class CustomImageFeatureEfficientNet(nn.Module):
    def __init__(self, feature_input_size, pretrained=True):
        super(CustomImageFeatureEfficientNet, self).__init__()

        # Load a pretrained EfficientNet model for image feature extraction (EfficientNet-B0 in this case)
        efficientnet = models.efficientnet_b0(pretrained=pretrained)  # You can change this to another EfficientNet version like B1 or B7
        self.efficientnet = nn.Sequential(*list(efficientnet.children())[:-1])  # Remove the final classification layer

        # The output of EfficientNet-B0's last conv layer is 1280-dimensional
        self.fc_image = nn.Linear(1280, 512)  # Reduce dimension to match your custom architecture

        # Fully connected layer for metadata (feature data)
        self.fc_metadata = nn.Linear(feature_input_size, 128)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.5)  # 50% dropout

        # Final fully connected layer for binary classification (combined image + feature input)
        self.fc_combined = nn.Linear(512 + 128, 1)  # For binary classification

    def forward(self, image, metadata):
        # Forward pass for the image through EfficientNet (without the final classification layer)
        x = self.efficientnet(image)  # EfficientNet feature extraction
        x = x.view(x.size(0), -1)  # Flatten the EfficientNet output
        image_features = F.relu(self.fc_image(x))

        # Process metadata (feature data)
        metadata_features = F.relu(self.fc_metadata(metadata))

        # Ensure the batch sizes are consistent
        assert image_features.shape[0] == metadata_features.shape[0], \
            f"Batch sizes do not match! Image batch size: {image_features.shape[0]}, Metadata batch size: {metadata_features.shape[0]}"

        # Concatenate image features and metadata features
        combined_features = torch.cat((image_features, metadata_features), dim=1)

        # Dropout and final classification layer
        combined_features = self.dropout(combined_features)
        output = self.fc_combined(combined_features)

        # If you're using BCELoss, uncomment the next line to apply sigmoid
        output = torch.sigmoid(output)

        return output


### Model Training
#### This cell contains the score function as well as the training and validation loop. The score function calculates the partial AUC-above-TPR, a key evaluation metric that focuses on the model's performance in high true positive rate regions. This is critical for ensuring that malignant lesions are correctly classified.

#### During the model training process, I implemented early stopping and model checkpointing to enhance performance and prevent overfitting. At each epoch, the model's validation loss is tracked, and if it achieves the lowest validation loss observed so far, the model is saved as the best model. This best-performing version will be used for later deployment, ensuring that only the most optimal and generalizable model is selected for real-world use. By doing so, I can ensure that the final deployed model achieves a balance between bias and variance while maintaining strong predictive performance on unseen data.

In [ ]:
# Function to compute partial AUC-above-TPR
def score(solution: np.array, submission: np.array, min_tpr: float = 0.80) -> float:
    """
    Compute the partial AUC by focusing on a specific range of true positive rates (TPR).
    
    Args:
        solution (np.array): Ground truth binary labels.
        submission (np.array): Model predictions.
        min_tpr (float): Minimum true positive rate to calculate partial AUC.
    
    Returns:
        float: The calculated partial AUC.
    
    Raises:
        ValueError: If the min_tpr is not within a valid range.
    """
    # Rescale the target to handle sklearn limitations and flip the predictions
    v_gt = abs(solution - 1)
    v_pred = -1.0 * submission
    max_fpr = abs(1 - min_tpr)

    # Compute ROC curve using sklearn
    fpr, tpr, _ = roc_curve(v_gt, v_pred)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError(f"Expected min_tpr in range [0, 1), got: {min_tpr}")
    
    # Interpolate for partial AUC
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    
    return partial_auc

In [ ]:


# Training and validation loop function
def train_and_validate(
    model: nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    val_dataloader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device,
    best_model_path: str,
    early_stopping_patience: int = 5,
    min_tpr: float = 0.80
    
) -> nn.Module:
    """
    Train and validate a PyTorch model with early stopping, AUROC, partial AUC, and error handling.

    Args:
        model (nn.Module): The model to be trained and validated.
        train_dataloader (torch.utils.data.DataLoader): Dataloader for training data.
        val_dataloader (torch.utils.data.DataLoader): Dataloader for validation data.
        criterion (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer to update the model.
        epochs (int): Number of training epochs.
        device (torch.device): The device (CPU or GPU) to use.
        early_stopping_patience (int): Early stopping patience.
        min_tpr (float): The minimum true positive rate for calculating partial AUC.
    
    Returns:
        nn.Module: The trained model.
    """
    # Initialize tracking variables
    best_val_loss = float('inf')
    best_epoch = 0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    early_stopping_counter = 0

    # Start the training and validation loop
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        
        # Training phase
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0
        all_train_labels = []
        all_train_probs = []

        progress_bar = tqdm(train_dataloader, desc=f'Training Epoch {epoch + 1}')

        try:
            # Loop through the training batches
            for i, (image, metadata, labels) in enumerate(progress_bar):
                image, metadata, labels = image.to(device), metadata.to(device), labels.float().to(device)
                labels = labels.unsqueeze(1)  # Adjust labels to have the right shape for binary classification

                optimizer.zero_grad()

                # Forward pass
                probs = model(image, metadata)

                if probs.shape != labels.shape:
                    raise ValueError(f"Shape mismatch: Predictions shape {probs.shape} does not match labels shape {labels.shape}")

                # Calculate loss and backpropagate
                loss = criterion(probs, labels)
                loss.backward()
                optimizer.step()

                # Update running loss
                running_train_loss += loss.item()

                # Store labels and predictions for accuracy calculations
                all_train_labels.extend(labels.cpu().detach().numpy())
                all_train_probs.extend(probs.cpu().detach().numpy())

                # Calculate binary predictions for training accuracy
                predicted_train = (probs >= 0.5).float()
                total_train += labels.size(0)
                correct_train += (predicted_train == labels).sum().item()

                # Update progress bar
                progress_bar.set_postfix(train_loss=running_train_loss / (i + 1))

            # Calculate training accuracy and loss
            train_accuracy = 100 * correct_train / total_train
            train_losses.append(running_train_loss / len(train_dataloader))
            train_accuracies.append(train_accuracy)

        except ValueError as ve:
            print(f"Error during training loop: {ve}")
            break

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct = 0
        total = 0
        all_labels = []
        all_probs = []

        progress_bar = tqdm(val_dataloader, desc=f'Validating Epoch {epoch + 1}')

        with torch.no_grad():
            try:
                # Loop through the validation batches
                for i, (images, metadata, labels) in enumerate(progress_bar):
                    images, metadata, labels = images.to(device), metadata.to(device), labels.float().to(device)
                    labels = labels.unsqueeze(1)

                    probs = model(images, metadata)

                    loss = criterion(probs, labels)
                    running_val_loss += loss.item()

                    all_labels.extend(labels.cpu().detach().numpy())
                    all_probs.extend(probs.cpu().detach().numpy())

                    # Calculate binary predictions for validation accuracy
                    predicted = (probs >= 0.5).float()
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    progress_bar.set_postfix(val_loss=running_val_loss / (i + 1))

                val_accuracy = 100 * correct / total
                val_loss = running_val_loss / len(val_dataloader)
                val_accuracies.append(val_accuracy)
                val_losses.append(val_loss)

                # Calculate AUROC
                try:
                    valid_auroc = roc_auc_score(all_labels, all_probs)
                except ValueError as ve:
                    print(f"AUROC Calculation Error: {ve}")
                    valid_auroc = 0.0

                # Calculate partial AUC-above-TPR
                try:
                    partial_auroc = score(np.array(all_labels), np.array(all_probs), min_tpr=min_tpr)
                except ValueError as ve:
                    print(f"Partial AUC Calculation Error: {ve}")
                    partial_auroc = 0.0

                print(f'Epoch [{epoch}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_loss:.4f}, '
                      f'Val Accuracy: {val_accuracy:.2f}%, Val AUROC: {valid_auroc:.4f}, Partial AUROC: {partial_auroc:.4f}')    

                # Early stopping based on validation loss
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_epoch = epoch + 1
                    early_stopping_counter = 0
                    torch.save(model.state_dict(), best_model_path)
                else:
                    early_stopping_counter += 1

                if early_stopping_counter >= early_stopping_patience:
                    print(f"Early stopping triggered at epoch {epoch}")
                    break

            except Exception as e:
                print(f"Error during validation loop: {e}")
                break

    print(f"Best Epoch: {best_epoch}, Best Validation Loss: {best_val_loss:.4f}")
    print('Training Complete')

    # Plot training and validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

    # Plot training and validation accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

    # Generate classification report
    try:
        print("Classification Report:")
        print(classification_report(all_labels, (np.array(all_probs) >= 0.5).astype(int), target_names=['Class 0', 'Class 1']))
    except Exception as e:
        print(f"Error generating classification report: {e}")

    return model



### Ready DataLoader for training

In [ ]:
# Initialize the dataset
CNN_train_dataset = MultiInputDataset(hdf5_file='../data/raw/train_images.hdf5', csv_file='../data/processed/processed-train-metadata1.csv', transform=get_train_transform(resize_size=(128,128)))
CNN_val_dataset = MultiInputDataset(hdf5_file='../data/raw/validation_image.hdf5', csv_file='../data/processed/processed-validation-metadata1.csv', transform=get_normal_transform(resize_size=(128,128)))
# Create a DataLoader
CNN_train_dataloader = DataLoader(CNN_train_dataset,  batch_size=64, shuffle=True)
CNN_val_dataloader = DataLoader(CNN_val_dataset,  batch_size=64, shuffle=True)

In [ ]:
# Initialize the dataset
resnet_train_dataset = MultiInputDataset(hdf5_file='../data/raw/train_images.hdf5', csv_file='../data/processed/processed-train-metadata1.csv', transform=get_train_transform(resize_size=(225,225)))
resnet_val_dataset = MultiInputDataset(hdf5_file='../data/raw/validation_image.hdf5', csv_file='../data/processed/processed-validation-metadata1.csv', transform=get_normal_transform(resize_size=(225,225)))
# Create a DataLoader
resnet_train_dataloader = DataLoader(resnet_train_dataset,  batch_size=64, shuffle=True)
resnet_val_dataloader = DataLoader(resnet_val_dataset,  batch_size=64, shuffle=True)

In [ ]:
# Initialize the dataset
effnet_train_dataset = MultiInputDataset(hdf5_file='../data/raw/train_images.hdf5', csv_file='../data/processed/processed-train-metadata1.csv', transform=get_train_transform(resize_size=(224,224)))
effnet_val_dataset = MultiInputDataset(hdf5_file='../data/raw/validation_image.hdf5', csv_file='../data/processed/processed-validation-metadata1.csv', transform=get_normal_transform(resize_size=(224,224)))
# Create a DataLoader
effnet_train_dataloader = DataLoader(effnet_train_dataset,  batch_size=64, shuffle=True)
effnet_val_dataloader = DataLoader(effnet_val_dataset,  batch_size=64, shuffle=True)

## Hyperparameter Tuning 
### Model 1

In [ ]:
model1 = CustomImageFeatureCNN2(feature_input_size=9)  # Assuming 9 features for metadata
model1.to(device)
# Initialize optimizer
optimizer = optim.Adam(model1.parameters(), lr=0.001)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
best_model_path = "best_model1.pth"

In [ ]:
train_and_validate(model1,CNN_train_dataloader, CNN_val_dataloader, criterion, optimizer, epochs, device ,best_model_path)

## Model 2

In [ ]:
model2 = CustomImageFeatureCNN2(feature_input_size=9)  # Assuming 9 features for metadata
model2.to(device)
# Initialize optimizer
optimizer = optim.SGD(model2.parameters(), lr=0.001)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
best_model_path = "best_model2.pth"

In [ ]:
train_and_validate(model2,CNN_train_dataloader, CNN_val_dataloader, criterion, optimizer, epochs, device,best_model_path )

## Model 3

In [ ]:
model3 = CustomImageFeatureCNN2(feature_input_size=9)  # Assuming 9 features for metadata
model3.to(device)
# Initialize optimizer
optimizer = optim.SGD(model3.parameters(), lr=0.0001,weight_decay=1e-4)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
batch_size = 32
best_model_path = "best_model3.pth"

In [ ]:
CNN_train_dataloader = DataLoader(CNN_train_dataset, batch_size=batch_size, shuffle=True)
CNN_val_dataloader = DataLoader(CNN_val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_and_validate(model3,CNN_train_dataloader, CNN_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

## Model 4

In [ ]:
model4 = CustomImageFeatureResNet(feature_input_size=9)  # Assuming 9 features for metadata
model4.to(device)
# Initialize optimizer
optimizer = optim.Adam(model4.parameters(), lr=0.001)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
best_model_path = "best_model4.pth"

In [ ]:
train_and_validate(model4,resnet_train_dataloader, resnet_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

## Model 5

In [ ]:
model5 = CustomImageFeatureResNet(feature_input_size=9)  # Assuming 9 features for metadata
model5.to(device)
# Initialize optimizer
optimizer = optim.SGD(model5.parameters(), lr=0.001)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
best_model_path = "best_model5.pth"

In [ ]:
train_and_validate(model5,resnet_train_dataloader, resnet_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

## Model 6

In [ ]:
model6 = CustomImageFeatureResNet(feature_input_size=9)  # Assuming 9 features for metadata
model6.to(device)
# Initialize optimizer
optimizer = optim.SGD(model6.parameters(), lr=0.0001,weight_decay=1e-4)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
batch_size = 32
best_model_path = "best_model6.pth"

In [ ]:
resnet_train_dataloader = DataLoader(resnet_train_dataset, batch_size=batch_size, shuffle=True)
resnet_val_dataloader = DataLoader(resnet_val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_and_validate(model6,resnet_train_dataloader, resnet_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

## Model 7

In [ ]:
model7 =  CustomImageFeatureEfficientNet(feature_input_size=9)  # Assuming 9 features for metadata
model7.to(device)
# Initialize optimizer
optimizer = optim.Adam(model7.parameters(), lr= 1.1621608010269284e-05)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
batch_size = 16
best_model_path = "best_model7.pth"




In [ ]:
effnet_train_dataloader = DataLoader(effnet_train_dataset,  batch_size=batch_size, shuffle=True)
effnet_val_dataloader = DataLoader(effnet_val_dataset,  batch_size=batch_size, shuffle=True)

In [ ]:
train_and_validate(model7,effnet_train_dataloader, effnet_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

## Model 8

In [ ]:
model8 = CustomImageFeatureEfficientNet(feature_input_size=9)  # Assuming 9 features for metadata
model8.to(device)
# Initialize optimizer
optimizer = optim.SGD(model8.parameters(), lr=0.01)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
best_model_path = "best_model8.pth"

In [ ]:
train_and_validate(model8,effnet_train_dataloader, effnet_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

## Model 9

In [ ]:
model9 = CustomImageFeatureEfficientNet(feature_input_size=9)  # Assuming 9 features for metadata
model9.to(device)
# Initialize optimizer
optimizer = optim.Adam(model9.parameters(), lr=0.001)
# Define the loss function with the class weights
criterion = nn.BCELoss()  # Binary classification loss
# Set the number of epochs
epochs = 20
batch_sizes = 16
best_model_path = "best_model9.path"

In [ ]:
train_and_validate(model9,effnet_train_dataloader, effnet_val_dataloader, criterion, optimizer, epochs, device, best_model_path )

# Select Winning Model 
#### Based on the performance metrics of the 9 models, Model 7 has been selected as the winning model. This decision was made after evaluating each model's performance on key metrics such as accuracy, AUROC, partial AUC, loss, precision, and recall.

#### The next step is to evaluate Model 7 on the test data, which contains unseen data that was not used during training or validation. This step is essential to assess the model's ability to generalize to new, real-world cases.

In [ ]:
effnet_test_dataset = MultiInputDataset(hdf5_file='../data/raw/test_image.hdf5', csv_file='../data/processed/processed-test-metadata1.csv', transform=get_normal_transform(resize_size=(224,224)))
# Create a DataLoader
effnet_test_dataloader = DataLoader(effnet_test_dataset,  batch_size=64, shuffle=True)

In [ ]:
final_model = CustomImageFeatureEfficientNet(9)
final_model_path = "best_model7.pth"
final_model.load_state_dict(torch.load(final_model_path, map_location=torch.device('cpu')))

final_model.eval()
all_labels, all_probs = [], []
with torch.no_grad():
    for images, metadata, labels in effnet_test_dataloader:
        images, metadata, labels = images.to(device), metadata.to(device), labels.float().to(device).unsqueeze(1)
        probs = final_model(images,metadata)
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        predicted = (probs > 0.5).float()

    partial_auroc=score(np.array(all_labels),np.array(all_probs))
    print(f'The partial auroc of the final model on the test image is {partial_auroc}')
    print(classification_report(all_labels, (np.array(all_probs) >= 0.5).astype(int), target_names=['Class 0', 'Class 1']))
        
        
        

#### As I expected, comparing the performance metrics on the test data versus the validation data for Model 7 reveals an improvement in the recall and F1-score for Class 1. This is a significant observation because it indicates that the model generalizes well to unseen data, which is crucial for real-world applications. Additionally, the partial AUC-above-TPR also shows improvement compared to the best epoch of Model 7 during validation. This suggests that the model performs better in capturing true positive malignant cases in regions of high true positive rates (TPR), which aligns with the primary goal of detecting malignant skin lesions effectively. These results demonstrate that the model is not overfitting to the validation set and is capable of making accurate predictions on new, unseen data.